# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

In [9]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
# MNIST: Modified National Institute of Standards and Technology
# Handwritten number database
# each x is a matrix of black & white pixel values
mnist = tf.keras.datasets.mnist # module with only fxn `load_data()`
loded = mnist.load_data()
#print(loded)
(x_train, y_train), (x_test, y_test) = loded
#print("x_train")
#print(x_train[1])
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model by stacking layers. Choose an optimizer and loss function for training:
* `keras.Sequential(layers="<layers>", name="<name>")` defines the linear stack of layers
    - inherits from `Model`

In [10]:
# baby way of creating architecture sequence:
tmp_model = tf.keras.models.Sequential()
tmp_model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
tmp_model.add(tf.keras.layers.Dense(128, activation='relu')) # as in: Densely connected NN layer
tmp_model.add(tf.keras.layers.Dropout(0.2))
tmp_model.add(tf.keras.layers.Dense(10))

# succinct way of doing it:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10),
])

For each example **the model returns** a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.
* `logits`: $\vec{z}$, non-normalized prediction outputs from previous layer
    - what you pass to a softmax after normalization (in a multi-class classification problem)
* `log-odds`: log of the odds (probability)
    - $logodds = ln(p)$
    - $logodds = inverse(sigmoid())$
        - $inverse()$: the function that "reverses" the original function

In [25]:
                     # first training ex. - a picture of shape (1, 28, 28)
predictions = model(x_train[:1]).numpy() # fwd pass on training ex. 1
                                  # numpy so it outputs a nparray and not a tf (eager)tensor
predictions

array([[ 0.14077179, -0.3158629 , -0.1788995 , -0.3343001 , -0.6662841 ,
        -0.358055  , -0.6271671 ,  0.8809456 ,  0.3012115 , -0.06284317]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 
* remember softmax outputs vector of probabilities, each one is the prob. that the input is a certain class
    - $\vec{z}$ (`logits`) is the output from previous layer
    - $\hat{y}_i = \frac{exp(z_i)}{\sum_{i} exp(z_i)}$

In [26]:
tf.nn.softmax(predictions).numpy()

array([[0.11648057, 0.07378015, 0.08461006, 0.07243232, 0.0519701 ,
        0.07073198, 0.0540433 , 0.24417819, 0.13675132, 0.0950221 ]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.
* `from_logits`: whether inputted `y_pred` is a logits tensor

In [29]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [31]:
loss_fn(y_train[:1], predictions).numpy() # Loss(y, ŷ)

2.6488576

In [32]:
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy'] # metrics that you want to evaluate during tran & test, we usually want accuracy
)

The `Model.fit` method adjusts the model parameters to minimize the loss: 
* This is what actually does the training, set # of training epochs

In [33]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 43us/sample - loss: 0.2958 - accuracy: 0.9137
Epoch 2/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.1464 - accuracy: 0.9564
Epoch 3/5
60000/60000 [==============================] - 2s 41us/sample - loss: 0.1075 - accuracy: 0.9676
Epoch 4/5
60000/60000 [==============================] - 3s 42us/sample - loss: 0.0863 - accuracy: 0.9731
Epoch 5/5
60000/60000 [==============================] - 3s 49us/sample - loss: 0.0734 - accuracy: 0.9776


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".
* returns loss value, and metrics specified in model.compile()
* `verbose=<num>`: arg that affects output
    - `0`: no output printed
    - `1`: print progress as it goes
    - `2`: print final progress

In [37]:
model.evaluate(x_test, y_test, verbose=2)

10000/10000 [==============================] - 0s 24us/sample - loss: 0.0692 - accuracy: 0.9770


[0.06918453295819928, 0.977]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [38]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [39]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.2788648e-07, 3.4552947e-08, 8.1315142e-05, 1.9703982e-04,
        3.2309730e-10, 4.7943183e-07, 1.1555588e-12, 9.9971944e-01,
        1.5433955e-07, 1.1948903e-06],
       [3.1749366e-06, 7.4684838e-05, 9.9989998e-01, 1.7537337e-05,
        3.0660560e-13, 1.4134371e-06, 4.0553584e-07, 1.3627312e-12,
        2.7374963e-06, 2.8960217e-12],
       [1.1450998e-06, 9.9868160e-01, 4.3684643e-04, 2.2108181e-05,
        1.6181462e-05, 2.9788603e-06, 8.7013350e-06, 6.6585722e-04,
        1.6367562e-04, 9.0791985e-07],
       [9.9984384e-01, 6.3246413e-09, 4.9352609e-05, 2.5079412e-08,
        1.8421578e-07, 1.2822499e-06, 3.2470452e-05, 5.8358812e-05,
        3.2305419e-07, 1.4079625e-05],
       [8.7099255e-07, 5.4560367e-10, 1.6662150e-05, 2.5365885e-08,
        9.9474949e-01, 1.0487025e-06, 4.8035936e-06, 5.4399151e-04,
        3.0579133e-06, 4.6801562e-03]], dtype=float32)>